In [2]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

True

In [3]:
import torch
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from openai import OpenAI

# 경로/모델 설정
DB_DIR = "./chroma_db"
COLLECTION_NAME = "qna_collection"
EMBED_MODEL = "BAAI/bge-m3"
OPENAI_MODEL = "gpt-4o-mini"
TOP_K = 5


embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

# 크로마 벡터스토어 로드
vs = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=DB_DIR,
    embedding_function=embeddings,
)

# OpenAI 클라이언트
oclient = OpenAI()

print("Vector store loaded. Count:", vs._collection.count())

c:\Users\playdata\miniconda3\envs\final_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\playdata\AppData\Local\Temp\ipykernel_18932\3941828245.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


Vector store loaded. Count: 14858


In [4]:
retriever = vs.as_retriever(search_kwargs={"k": TOP_K})
test_docs = retriever.get_relevant_documents("BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?")
print("검색 결과 개수:", len(test_docs))
for i, d in enumerate(test_docs, 1):
    print(f"[{i}] source:", d.metadata.get("source") or d.metadata.get("sources"))
    print(d.page_content[:200].replace("\n"," "), "\n---")

C:\Users\playdata\AppData\Local\Temp\ipykernel_18932\4218759091.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_docs = retriever.get_relevant_documents("BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?")


검색 결과 개수: 5
[1] source: ["https://cloud.google.com/bigquery/docs/reference/rest"]
Q: BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요? A: 새로운 데이터셋을 생성하려면 다음의 API 메서드를 사용합니다: `insert` 메서드. 요청은 다음과 같이 구성됩니다:  ``` POST /bigquery/v2/projects/{projectId}/datasets ``` 여기서 `{projectId}`는 데이터셋을 생성할 프로젝 
---
[2] source: ["https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets/insert"]
Q: BigQuery에서 새로운 데이터셋을 생성하기 위한 HTTP 요청은 어떻게 구성하나요? A: 새로운 데이터셋을 생성하기 위한 HTTP 요청은 다음과 같이 구성됩니다:  ``` POST https://bigquery.googleapis.com/bigquery/v2/projects/{projectId}/datasets ``` 여기서 `{projectId} 
---
[3] source: ["https://cloud.google.com/bigquery/docs/reference/rest"]
Q: BigQuery에서 특정 데이터셋에 새로운 루틴을 생성하는 방법은 무엇인가요? A: 새로운 루틴을 생성하려면 다음의 REST API를 사용합니다: POST /bigquery/v2/projects/{projectId}/datasets/{datasetId}/routines 
---
[4] source: ["https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets"]
Q: BigQuery 데이터셋에 대한 태그를 설정하는 방법은 무엇인가요? A: 데이터셋에 태그(resourceTags)를 추가하려면, 데이터셋을 생성할 때 해당 필드를 사용하여 태그를 지정해

In [5]:
def _show_sources(meta) -> str:

    s = meta.get("source")
    if not s:
        s = meta.get("sources")
        if isinstance(s, list):
            s = ", ".join([str(x) for x in s])
    return s or "(no source)"

def build_messages(user_query: str, docs):
    lines = []
    for i, d in enumerate(docs, start=1):
        src = _show_sources(d.metadata or {})
        body = d.page_content[:1200]
        lines.append(f"[{i}] source={src}\n{body}")
    context_block = "\n\n".join(lines) if lines else "(no context)"

    system_prompt = (
        "너는 Google API 문서 기반 한국어 어시스턴트다. "
        "아래 제공된 근거 스니펫 안에서 확인 가능한 내용만 답하고, 모르면 모른다고 말해라. "
        "고유 식별자/API/메서드/필드/에러명/옵션명은 원문 영문을 괄호로 병기하고, "
        "코드 블록은 수정/번역하지 말고 원문 그대로 인용해라."
    )

    user_prompt = f"""사용자 질문:
{user_query}

[근거 스니펫]
{context_block}

요구사항:
- 스니펫에서 직접 확인 가능한 사실만 사용.
- 스니펫에서 확인 불가하거나 불확실하면 '문서에서 확인 불가'라고 명시.
- 답변은 단순 요약이 아니라, 사용자가 이해하기 쉽게 **조금 더 설명**을 붙여라.
- 답변 끝에는 '참고' 섹션을 추가하여, 본문에서 실제로 인용한 근거 번호와 해당 SOURCE URL만 나열하라. (다른 메타데이터는 제외)
- 코드 블록이 있으면 그대로 인용하고, 필요시 짧은 해설을 추가하라.
- 최종 출력은 한국어.
"""
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

def rag_answer(query: str, top_k: int = TOP_K):
    # 1) 검색
    retriever = vs.as_retriever(search_kwargs={"k": top_k})
    docs = retriever.get_relevant_documents(query)

    # 2) LLM 호출
    messages = build_messages(query, docs)
    resp = oclient.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        temperature=0.0,
    )
    answer = resp.choices[0].message.content
    return answer, docs

In [19]:
question = "BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?"
answer, evidences = rag_answer(question)

print("=== 질문 ===")
print(question)

print("\n=== 답변 ===")
print(answer.strip())

=== 질문 ===
BigQuery에서 새로운 데이터셋을 생성하는 방법은 무엇인가요?

=== 답변 ===
BigQuery에서 새로운 데이터셋을 생성하려면 `insert` 메서드를 사용해야 합니다. 이 메서드는 HTTP POST 요청을 통해 호출되며, 요청 URL은 다음과 같이 구성됩니다:

```
POST https://bigquery.googleapis.com/bigquery/v2/projects/{projectId}/datasets
```

여기서 `{projectId}`는 데이터셋을 생성할 프로젝트의 ID를 의미합니다. 이 요청을 통해 새로운 데이터셋이 해당 프로젝트 내에 생성됩니다. 데이터셋을 생성할 때는 추가적인 설정이나 속성을 포함할 수 있으며, 필요한 경우 태그를 지정할 수도 있습니다.

참고:
- [1] https://cloud.google.com/bigquery/docs/reference/rest
- [2] https://cloud.google.com/bigquery/docs/reference/rest/v2/datasets/insert


In [6]:
question = "Firebase Admin SDK를 사용하여 커스텀 토큰을 만들기 위해 서비스 계정 JSON 파일을 사용하는 방법은 무엇인가요?"
answer, evidences = rag_answer(question)

print("=== 질문 ===")
print(question)

print("\n=== 답변 ===")
print(answer.strip())

=== 질문 ===
Firebase Admin SDK를 사용하여 커스텀 토큰을 만들기 위해 서비스 계정 JSON 파일을 사용하는 방법은 무엇인가요?

=== 답변 ===
Firebase Admin SDK를 사용하여 커스텀 토큰을 만들기 위해 서비스 계정 JSON 파일을 사용하는 방법은 다음과 같습니다. 먼저, Firebase Admin SDK를 초기화해야 합니다. 이 과정에서 서비스 계정 JSON 파일을 사용하여 Admin SDK를 설정할 수 있습니다. 이렇게 하면 원격 API 호출 없이 로컬에서 커스텀 토큰을 만들고 서명할 수 있습니다.

다음은 Python에서 Admin SDK를 초기화하는 예시입니다:

```python
default_app = firebase_admin.initialize_app()
```

이 코드를 실행하면 Firebase Admin SDK가 초기화되고, 이후에 커스텀 토큰을 생성할 수 있는 준비가 완료됩니다. 서비스 계정 JSON 파일을 사용하면 애플리케이션의 보안성을 높이고, 코드에 민감한 정보를 포함할 필요가 없어집니다.

또한, 서비스 계정 ID를 명시적으로 지정하여 초기화할 수도 있습니다. 이 방법은 애플리케이션의 다양한 부분에서 일관성을 유지하는 데 도움이 됩니다. 예를 들어, Node.js에서는 다음과 같이 사용할 수 있습니다:

```javascript
initializeApp({
  serviceAccountId: 'my-client-id@my-project-id.iam.gserviceaccount.com',
```

이와 같은 방식으로 서비스 계정 ID를 사용하면 Google이 관리하는 환경에서 실행할 때 이 ID의 키가 토큰 서명에 사용됩니다.

참고:
1. [1] https://firebase.google.com/docs/auth/admin/create-custom-tokens?hl=ko
2. [3] https://firebase.google.com/docs/auth/admin/create-custom-token

In [7]:
question = "People API를 사용하기 위해 quickstart.py 파일에 포함해야 할 기본 코드는 무엇인가요?"
answer, evidences = rag_answer(question)

print("=== 질문 ===")
print(question)

print("\n=== 답변 ===")
print(answer.strip())

=== 질문 ===
People API를 사용하기 위해 quickstart.py 파일에 포함해야 할 기본 코드는 무엇인가요?

=== 답변 ===
People API를 사용하기 위해 `quickstart.py` 파일에 포함해야 할 기본 코드는 다음과 같습니다. 이 코드는 Google API에 접근하기 위한 인증 및 기본 설정을 포함하고 있습니다.

```python
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/contacts.readonly"]

def main():
  """Shows basic usage of the People API.
  Prints the name of the first 10 connections.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are n